In [2]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [3]:
grass_womens=hist_data_dog('grass','Womens',1,2,datefilter)
grass_mens=hist_data_dog('grass','Mens',1,2,datefilter)
temp=pd.concat([grass_womens,grass_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
mind=temp[temp['Fav_Odds']<1.3].copy()
mind=mind[mind['Fav_Odds']>1]
mind=mind[mind['Date']>='2024-01-01']

favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()



In [4]:
Fav_Loss_Count=temp[['Date','Fav','Fav_Loss_Count']].copy()
Fav_Loss_Count['Date'] = pd.to_datetime(Fav_Loss_Count['Date'])
latest_per_fav = Fav_Loss_Count[Fav_Loss_Count['Date'] == Fav_Loss_Count.groupby('Fav')['Date'].transform('max')]
latest_per_fav[['Fav','Fav_Loss_Count']].to_csv('Fav_Loss_Count.csv')
Dog_Win_Count=temp[['Date','Dog','Dog_Win_Count']].copy()
Dog_Win_Count['Date'] = pd.to_datetime(Dog_Win_Count['Date'])
latest_per_dog = Dog_Win_Count[Dog_Win_Count['Date'] == Dog_Win_Count.groupby('Dog')['Date'].transform('max')]
latest_per_dog[['Dog','Dog_Win_Count']].to_csv('Dog_Win_Count.csv')

In [ ]:
temp
stake=100
date_filter=temp[temp['Date']>='2024-01-01']
oddsfilter=date_filter[date_filter['Dog_Odds']>2.2].copy()
oddsfilter=oddsfilter[oddsfilter['Fav_Odds']>1]
#oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>1]
#oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>1]
oddsfilter=oddsfilter[oddsfilter['fav_rank_high']>oddsfilter['dog_rank_high']]
oddsfilter=oddsfilter[oddsfilter['fav_rank']>oddsfilter['dog_rank']]

oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter.to_clipboard(index=False)

Total Profit: -792.0 177 0.327683615819209


In [52]:
import pandas as pd

# Ensure date is datetime and sorted
oddsfilter['Date'] = pd.to_datetime(oddsfilter['Date'])
oddsfilter = oddsfilter.sort_values('Date').reset_index(drop=True)

# Initialize bankroll tracking
initial_bankroll = 1000
daily_bankrolls = {}
current_bankroll = initial_bankroll

stakes = []
profits = []
bankrolls = []

# Group by date
for date, day_df in oddsfilter.groupby('Date'):
    stake = 0.1 * current_bankroll  # stake same for all matches on this day

    day_profits = []

    # Compute profit for each row in the day
    for _, row in day_df.iterrows():
        if row['DogWin'] == 1:
            profit = row['Dog_Odds'] * stake - stake
        else:
            profit = -stake
        day_profits.append(profit)

        # Append values for each row
        stakes.append(stake)
        profits.append(profit)
        bankrolls.append(current_bankroll)

    # Update bankroll after all bets for the day
    current_bankroll += sum(day_profits)

# Assign columns
oddsfilter['Stake'] = stakes
oddsfilter['Profit'] = profits
oddsfilter['Bankroll'] = bankrolls
oddsfilter.to_clipboard()

In [ ]:
temp
date_filter=temp[temp['Date']>'2024-06-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
#oddsfilter=date_filter[date_filter['Dog_Odds']<2]
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>1]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>1]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Fav_WinPct_Last10']<0.5]
#oddsfilter=oddsfilter[oddsfilter['Dog_WinPct_Last10']>oddsfilter['Fav_WinPct_Last10']]
#oddsfilter=oddsfilter[oddsfilter['Fav_WinPct_Last10']<0.5]
oddsfilter=oddsfilter[oddsfilter['dog_percent']>0.5]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter.to_csv('_grassResults_Mens.csv')

Total Profit: 3565.0 187 0.5882352941176471


In [304]:
#from sqlalchemy import text

#with devengine.connect() as conn:
#    conn.execute(text('DELETE FROM results_hard_1 WHERE Date = "2025-05-23"'))
#    conn.commit()

In [ ]:
date_filter = pd.read_pickle(r'.\grass_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_loss = pd.read_csv(r'.\Fav_Loss_Count.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_loss_subset = fav_loss[['Fav', 'Fav_Loss_Count']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Win_Count']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_loss_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
oddsfilter=date_filter[date_filter['Fav_Odds']>1.4].copy()

oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter[['Dog','Dog_Odds','Fav']].to_clipboard(index=False)

In [ ]:
date_filter = pd.read_pickle(r'.\grass_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_loss = pd.read_csv(r'.\Fav_Loss_Count.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_loss_subset = fav_loss[['Fav', 'Fav_Loss_Count']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Win_Count']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_loss_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
oddsfilter=oddsfilter[oddsfilter['dog_percent']>0.5]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter[['Dog','Dog_Odds','Fav']].to_clipboard(index=False)


In [ ]:
lowodds=pd.read_pickle(r'.\grass_Today')
lowodds=lowodds[lowodds['Fav_Odds']<1.3].copy()
lowodds=lowodds[lowodds['Fav_Odds']>1]
favs=pd.read_csv('Fav_grass.csv')
fav_list=favs['Name'].tolist()
lowodds=lowodds[(lowodds['Fav']).isin(fav_list)]
lowodds=lowodds[lowodds['dog_rank']>50]
lowodds

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
4,20:00,1.08,8.03,Elina Svitolina,Elina Svitolina,84%,25,Zeynep Sonmez,33%,6,...,0.7,0.3,0.64,0.37,0.0,0.00,0.6,0.2,5.0,1.0
11,21:00,1.02,19.97,Aryna Sabalenka,Aryna Sabalenka,89%,18,Kamilla Rakhimova,33%,6,...,0.8,0.4,0.68,0.37,0.6,0.33,0.8,0.4,5.0,11.0


In [ ]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)  
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

date_filter = pd.read_pickle(r'.\grass_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_loss = pd.read_csv(r'.\Fav_Loss_Count.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_loss_subset = fav_loss[['Fav', 'Fav_Loss_Count']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Win_Count']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_loss_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>2]
today=oddsfilter[oddsfilter['Sex']=='Mens'].copy()
today['Date']=date_string
today.to_sql('Preditions_grass',con=devengine,if_exists='append',index=False)

###############################################################################################
###############################################################################################

date_filter = pd.read_pickle(r'.\grass_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_loss = pd.read_csv(r'.\Fav_Loss_Count.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_loss_subset = fav_loss[['Fav', 'Fav_Loss_Count']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Win_Count']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_loss_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
oddsfilter=date_filter[date_filter['Fav_Odds']>1.4].copy()

oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
today=oddsfilter[oddsfilter['Sex']=='Womens'].copy()
today['Date']=date_string
today.to_sql('Preditions_grass',con=devengine,if_exists='append',index=False)

4

In [ ]:
predictions=pd.read_sql_query('Select Distinct * from Preditions_grass where date > "2025-05-22"',con=devengine)
results=temp

# Ensure date types
predictions['Date'] = pd.to_datetime(predictions['Date'])
results['Date'] = pd.to_datetime(results['Date'])

# Step 1: Initial merge on the exact date
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],
    on=['Date', 'Fav', 'Dog'], 
    how='left'
)

# Step 2: Get unmatched rows only
unmatched = merged_df[merged_df['Winner'].isna()].copy()

if not unmatched.empty:
    # Try matching with results where match happened one day after the prediction date
    results_shifted = results.copy()
    results_shifted['Date'] = results_shifted['Date'] - pd.Timedelta(days=1)

    fallback = pd.merge(
        unmatched.drop(columns=['Winner']),  # remove old (NaN) winner
        results_shifted[['Date', 'Fav', 'Dog', 'Winner']],
        on=['Date', 'Fav', 'Dog'],
        how='left'
    )

    # Only update original merged_df where Winner was missing AND fallback found a match
    merged_df.loc[merged_df['Winner'].isna(), 'Winner'] = fallback['Winner'].values


# merged_df now has best effort match: same day, or next day fallback
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.4].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['dog_percent']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Dog_Win_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Sex']=='Womens']
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Elo_Fav']==merged_df_filtered['Fav']]

merged_df_filtered['Profit'] = merged_df_filtered.apply(    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered


Total Profit: -7.000000000000057 9 0.4444444444444444


,Date,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,...,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count,Winner,Profit
2,2025-05-23,20:00,1.70,2.09,Solana Sierra,Solana Sierra,55%,11,Xiyu Wang,25%,...,0.00,0.00,0.68,0.32,0.6,0.6,2,5,Solana Sierra,-100.0
9,2025-05-23,1:30,1.71,2.12,Paula Badosa,Paula Badosa,75%,8,Liudmila Samsonova,44%,...,0.68,0.36,0.81,0.31,0.6,0.6,1,3,Liudmila Samsonova,112.0
11,2025-05-23,20:00,1.57,2.32,Taylor Townsend,Taylor Townsend,50%,2,Daria Saville,50%,...,0.64,0.29,0.68,0.34,0.4,0.4,1,1,Daria Saville,132.0
12,2025-05-23,21:30,1.47,2.56,Julia Riera,Miriam Bulgaru,71%,14,Miriam Bulgaru,30%,...,0.00,0.00,0.00,0.00,0.8,0.4,6,5,Julia Riera,-100.0
14,2025-05-23,23:00,1.75,2.01,Veronika Erjavec,Veronika Erjavec,57%,14,Lucrezia Stefanini,50%,...,0.00,0.00,0.57,0.28,0.8,0.6,5,2,Lucrezia Stefanini,101.0
19,2025-05-24,0:25,1.86,1.93,Jaqueline Cristian,Camila Osorio,56%,18,Camila Osorio,61%,...,NaN,NaN,0.57,0.37,0.8,0.6,3,2,Jaqueline Cristian,-100.0
20,2025-05-24,23:00,1.72,2.09,Maya Joint,Maya Joint,75%,8,Jaqueline Cristian,77%,...,NaN,NaN,NaN,NaN,0.8,0.8,1,3,Maya Joint,-100.0
21,2025-05-24,2:45,1.53,2.48,Danielle Collins,Danielle Collins,75%,20,Liudmila Samsonova,46%,...,NaN,NaN,0.81,0.31,0.8,0.6,2,4,Liudmila Samsonova,148.0
22,2025-05-24,23:00,1.45,2.74,Elena Rybakina,Elena Rybakina,81%,16,Liudmila Samsonova,50%,...,0.78,0.30,0.81,0.31,0.8,0.6,3,4,Elena Rybakina,-100.0


In [ ]:
#df=pd.read_clipboard()
#df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
#df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
#df.to_sql('Preditions_grass',con=devengine,if_exists='replace',index=False)



#pd.read_sql_query('Select Distinct * from Preditions_grass',con=devengine).to_clipboard(index=False)

In [311]:
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.6].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['dog_percent']>0.5]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Dog_Win_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Sex']=='Mens']
merged_df_filtered=merged_df_filtered[merged_df_filtered['Elo_Fav']==merged_df_filtered['Dog']]

merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered.tail(5)

Total Profit: 29.99999999999997 2 0.5


,Date,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,...,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count,Winner,Profit
0,2025-05-23,3:00,1.61,2.30,Taylor Fritz,Hubert Hurkacz,56%,25,Hubert Hurkacz,62%,...,0.84,0.24,0.91,0.18,0.4,0.8,5,6,Hubert Hurkacz,130.0
5,2025-05-23,21:00,1.85,1.86,Nerman Fatic,Mili Poljicak,75%,20,Mili Poljicak,86%,...,0.00,0.00,0.54,0.21,0.8,0.8,7,7,Nerman Fatic,-100.0
